In [16]:
import sys
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.xception import Xception, preprocess_input
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report, roc_curve
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import class_weight
import keras
from keras import backend as K
from keras.models import Sequential, Model
from keras.layers import Input,Dense, Dropout, Activation, Flatten, Convolution2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers.core import Dropout
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt

In [17]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(288, 288, 3)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 286, 286, 32)      896       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 284, 284, 64)      18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 142, 142, 64)      0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 142, 142, 64)      0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 1290496)           0         
_________________________________________________________________
dense_3 (Dense)              (None, 64)                82591808  
_________________________________________________________________
dropout_4 (Dropout)          (None, 64)               

In [18]:
early_stopping = EarlyStopping(monitor='val_loss', patience=50, verbose=1)
callbacks = [early_stopping]

In [19]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,  
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.1,
                                   fill_mode='nearest',
                                   horizontal_flip = False)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [20]:
training_set = train_datagen.flow_from_directory('data/19_class2/train',
                                                 target_size = (288, 288),
                                                 batch_size = 16,
                                                 class_mode = 'categorical', shuffle=True)

Found 7955 images belonging to 5 classes.


In [21]:
test_set = test_datagen.flow_from_directory('data/19_class2/test',
                                            target_size = (288, 288),
                                            batch_size = 16,
                                            class_mode = 'categorical')

Found 968 images belonging to 5 classes.


In [22]:
val_set = test_datagen.flow_from_directory('data/19_class2/val',
                                            target_size = (288, 288),
                                            batch_size = 16,
                                            class_mode = 'categorical')

Found 1561 images belonging to 5 classes.


In [25]:
class_weights = class_weight.compute_class_weight(
               'balanced',
                np.unique(training_set.classes), 
                training_set.classes)

In [27]:
model.fit_generator(training_set, epochs=10, class_weight=class_weights,  validation_data = test_set, verbose=True, callbacks=callbacks, shuffle=True)

Epoch 1/10
498/498 [==============================] - 1843s 4s/step - loss: 0.9849 - accuracy: 0.6333 - val_loss: 1.0383 - val_accuracy: 0.6736
Epoch 2/10
498/498 [==============================] - 1841s 4s/step - loss: 0.9646 - accuracy: 0.6393 - val_loss: 1.2206 - val_accuracy: 0.6374
Epoch 3/10
498/498 [==============================] - 1849s 4s/step - loss: 0.9429 - accuracy: 0.6478 - val_loss: 0.7554 - val_accuracy: 0.6312
Epoch 4/10
498/498 [==============================] - 1848s 4s/step - loss: 0.9331 - accuracy: 0.6471 - val_loss: 1.2609 - val_accuracy: 0.6777
Epoch 5/10
498/498 [==============================] - 1848s 4s/step - loss: 0.9041 - accuracy: 0.6621 - val_loss: 0.7249 - val_accuracy: 0.7180
Epoch 6/10
498/498 [==============================] - 1849s 4s/step - loss: 0.8889 - accuracy: 0.6645 - val_loss: 1.2034 - val_accuracy: 0.7262
Epoch 7/10
498/498 [==============================] - 1846s 4s/step - loss: 0.8563 - accuracy: 0.6832 - val_loss: 0.5849 - val_accuracy:

In [28]:
model.save('MK_19class2.h5')

In [29]:
Y_pred = model.predict_generator(val_set)
y_pred = np.argmax(Y_pred, axis=1)

In [30]:
print(classification_report(val_set.classes, y_pred))

              precision    recall  f1-score   support

           0       0.38      0.35      0.36       607
           1       0.05      0.01      0.02        76
           2       0.14      0.19      0.16       203
           3       0.08      0.08      0.08       154
           4       0.33      0.36      0.34       521

    accuracy                           0.29      1561
   macro avg       0.20      0.20      0.19      1561
weighted avg       0.29      0.29      0.29      1561



In [31]:
score = model.evaluate_generator(training_set, verbose=0)
print("Training Accuracy: ", score[1])

Training Accuracy:  0.7358893752098083


In [32]:
score = model.evaluate_generator(test_set, verbose=0)
print("Testing Accuracy: ", score[1])

Testing Accuracy:  0.7613636255264282


In [33]:
score = model.evaluate_generator(val_set, verbose=0)
print("Validating Accuracy: ", score[1])

Validating Accuracy:  0.7238949537277222
